#DA 204o Data Science in Practice

Project: Online News Popularity

Authors: Rajat Chaudhary, Sonu Goyal, Srinivas Shavukapu Kattegummula, Yuvaraj G
email: rajatc@iisc.ac.in, sonugoyal@iisc.ac.in, srinivassk@iisc.ac.in, yuvarajgopi@iisc.ac.in

# 00-Prerequisities
## Libraries Required and Import of all Libraries

In [ ]:
#Libraries Required and Import of all Libraries
import warnings
warnings.filterwarnings("ignore")
from pathlib import Path
import numpy as np
import pandas as pd
import math
import scipy.stats as stats

import matplotlib.pyplot as plt
import seaborn as sns

#Regression Algorithm models
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import StackingRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression


#Classification Algorithm models
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB





from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
from sklearn.model_selection import GroupKFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_squared_log_error
from sklearn.preprocessing import FunctionTransformer,OrdinalEncoder
from sklearn.model_selection import train_test_split, cross_val_score


from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.utils.class_weight import compute_sample_weight

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PowerTransformer
from sklearn.base import clone


from sklearn.inspection import permutation_importance

# 01-data-collection

In [ ]:
#01-data-collection
# # Paths
BASE_PATH = Path("/Users/yuvaraj/Desktop/IISc/01_Semester/DA 204o Data Science in Practice/Project/Input")
BASE_FILE = BASE_PATH / "OnlineNewsPopularity.csv"


In [3]:
df = pd.read_csv(BASE_FILE, low_memory=False)


In [4]:
print(df.head())
print("Shape of dataset",df.shape)

                                                 url   timedelta  \
0  http://mashable.com/2013/01/07/amazon-instant-...       731.0   
1  http://mashable.com/2013/01/07/ap-samsung-spon...       731.0   
2  http://mashable.com/2013/01/07/apple-40-billio...       731.0   
3  http://mashable.com/2013/01/07/astronaut-notre...       731.0   
4   http://mashable.com/2013/01/07/att-u-verse-apps/       731.0   

    n_tokens_title   n_tokens_content   n_unique_tokens   n_non_stop_words  \
0             12.0              219.0          0.663594                1.0   
1              9.0              255.0          0.604743                1.0   
2              9.0              211.0          0.575130                1.0   
3              9.0              531.0          0.503788                1.0   
4             13.0             1072.0          0.415646                1.0   

    n_non_stop_unique_tokens   num_hrefs   num_self_hrefs   num_imgs  ...  \
0                   0.815385         4.0     

Attribute Information:
     0. url:                           URL of the article (non-predictive)
     1. timedelta:                     Days between the article publication and the dataset acquisition (non-predictive)
     2. n_tokens_title:                Number of words in the title
     3. n_tokens_content:              Number of words in the content
     4. n_unique_tokens:               Rate of unique words in the content
     5. n_non_stop_words:              Rate of non-stop words in the content
     6. n_non_stop_unique_tokens:      Rate of unique non-stop words in the content
     7. num_hrefs:                     Number of links
     8. num_self_hrefs:                Number of links to other articles published by Mashable
     9. num_imgs:                      Number of images
    10. num_videos:                    Number of videos
    11. average_token_length:          Average length of the words in the content
    12. num_keywords:                  Number of keywords in the metadata
    13. data_channel_is_lifestyle:     Is data channel 'Lifestyle'?
    14. data_channel_is_entertainment: Is data channel 'Entertainment'?
    15. data_channel_is_bus:           Is data channel 'Business'?
    16. data_channel_is_socmed:        Is data channel 'Social Media'?
    17. data_channel_is_tech:          Is data channel 'Tech'?
    18. data_channel_is_world:         Is data channel 'World'?
    19. kw_min_min:                    Worst keyword (min. shares)
    20. kw_max_min:                    Worst keyword (max. shares)
    21. kw_avg_min:                    Worst keyword (avg. shares)
    22. kw_min_max:                    Best keyword (min. shares)
    23. kw_max_max:                    Best keyword (max. shares)
    24. kw_avg_max:                    Best keyword (avg. shares)
    25. kw_min_avg:                    Avg. keyword (min. shares)
    26. kw_max_avg:                    Avg. keyword (max. shares)
    27. kw_avg_avg:                    Avg. keyword (avg. shares)
    28. self_reference_min_shares:     Min. shares of referenced articles in Mashable
    29. self_reference_max_shares:     Max. shares of referenced articles in Mashable
    30. self_reference_avg_sharess:    Avg. shares of referenced articles in Mashable
    31. weekday_is_monday:             Was the article published on a Monday?
    32. weekday_is_tuesday:            Was the article published on a Tuesday?
    33. weekday_is_wednesday:          Was the article published on a Wednesday?
    34. weekday_is_thursday:           Was the article published on a Thursday?
    35. weekday_is_friday:             Was the article published on a Friday?
    36. weekday_is_saturday:           Was the article published on a Saturday?
    37. weekday_is_sunday:             Was the article published on a Sunday?
    38. is_weekend:                    Was the article published on the weekend?
    39. LDA_00:                        Closeness to LDA topic 0
    40. LDA_01:                        Closeness to LDA topic 1
    41. LDA_02:                        Closeness to LDA topic 2
    42. LDA_03:                        Closeness to LDA topic 3
    43. LDA_04:                        Closeness to LDA topic 4
    44. global_subjectivity:           Text subjectivity
    45. global_sentiment_polarity:     Text sentiment polarity
    46. global_rate_positive_words:    Rate of positive words in the content
    47. global_rate_negative_words:    Rate of negative words in the content
    48. rate_positive_words:           Rate of positive words among non-neutral tokens
    49. rate_negative_words:           Rate of negative words among non-neutral tokens
    50. avg_positive_polarity:         Avg. polarity of positive words
    51. min_positive_polarity:         Min. polarity of positive words
    52. max_positive_polarity:         Max. polarity of positive words
    53. avg_negative_polarity:         Avg. polarity of negative  words
    54. min_negative_polarity:         Min. polarity of negative  words
    55. max_negative_polarity:         Max. polarity of negative  words
    56. title_subjectivity:            Title subjectivity
    57. title_sentiment_polarity:      Title polarity
    58. abs_title_subjectivity:        Absolute subjectivity level
    59. abs_title_sentiment_polarity:  Absolute polarity level
    60. shares:                        Number of shares (target)

# 02-data-preprocessing

In [ ]:
#02-data-preprocessing
# Exploring the data for Number and Percentage of "NaN", Data type of column and Column values & its grouping

# User defined function to check above mentioned items for all columns
def value_counts_summary(df):
  summary = {}
  for col in df.columns:
    print("-"*80)
    null_count=df[col].isna().sum()
    null_percentage=null_count/len(df)*100
    print("Column : ",col,";   NA : ",null_count,";   NA_% : ",round(null_percentage,2), ";   Data_Type : ",df[col].dtype)
    summary[col] = df[col].value_counts(dropna=False)
    print(summary[col])
    print("-"*80)


value_counts_summary(df)

--------------------------------------------------------------------------------
Column :  url ;   NA :  0 ;   NA_% :  0.0 ;   Data_Type :  object
url
http://mashable.com/2013/01/07/amazon-instant-video-browser/                          1
http://mashable.com/2014/06/05/congress-bergdahl-swap/                                1
http://mashable.com/2014/06/05/barnes-noble-nook-samsung-galaxy-tab-4/                1
http://mashable.com/2014/06/05/beastie-boys-awarded-1-7-million-in-copyright-case/    1
http://mashable.com/2014/06/05/beats-world-cup-ad/                                    1
                                                                                     ..
http://mashable.com/2013/09/18/matthew-cordle-youtube-guilty/                         1
http://mashable.com/2013/09/18/memoir-app/                                            1
http://mashable.com/2013/09/18/microsoft-internet-explorer-11-update/                 1
http://mashable.com/2013/09/18/most-expensive-video-games

Observation: No NAN values are found in the model

In [6]:
#Remove whitespace in columns headers

def trim_column_names(df):
    # Print a sample before to see the issue
    print("Before Trimming :", df.columns[:].tolist())
    
    # The actual cleaning line: .str.strip() removes spaces from both ends
    df.columns = df.columns.str.strip()
    
    # Verify the change
    print("After Trimming: ", df.columns[:].tolist())
    
    return df

# Run the function
df = trim_column_names(df)

Before Trimming : ['url', ' timedelta', ' n_tokens_title', ' n_tokens_content', ' n_unique_tokens', ' n_non_stop_words', ' n_non_stop_unique_tokens', ' num_hrefs', ' num_self_hrefs', ' num_imgs', ' num_videos', ' average_token_length', ' num_keywords', ' data_channel_is_lifestyle', ' data_channel_is_entertainment', ' data_channel_is_bus', ' data_channel_is_socmed', ' data_channel_is_tech', ' data_channel_is_world', ' kw_min_min', ' kw_max_min', ' kw_avg_min', ' kw_min_max', ' kw_max_max', ' kw_avg_max', ' kw_min_avg', ' kw_max_avg', ' kw_avg_avg', ' self_reference_min_shares', ' self_reference_max_shares', ' self_reference_avg_sharess', ' weekday_is_monday', ' weekday_is_tuesday', ' weekday_is_wednesday', ' weekday_is_thursday', ' weekday_is_friday', ' weekday_is_saturday', ' weekday_is_sunday', ' is_weekend', ' LDA_00', ' LDA_01', ' LDA_02', ' LDA_03', ' LDA_04', ' global_subjectivity', ' global_sentiment_polarity', ' global_rate_positive_words', ' global_rate_negative_words', ' rate_

In [7]:
def check_column_types(df):
    print(f"{'COLUMN NAME':<30} | {'DATA TYPE CATEGORY':<20}")
    print("-" * 55)
    
    for col in df.columns:
        # Check if column is numeric (Integers or Floats)
        if pd.api.types.is_numeric_dtype(df[col]):
            col_type = "Numeric"
        # Check if column is object/string/categorical
        elif pd.api.types.is_string_dtype(df[col]) or pd.api.types.is_object_dtype(df[col]):
            col_type = "String / Object"
        else:
            col_type = "Other (Time/Bool)"
            
        print(f"{col:<30} | {col_type:<20}")

# Run the function
check_column_types(df)

COLUMN NAME                    | DATA TYPE CATEGORY  
-------------------------------------------------------
url                            | String / Object     
timedelta                      | Numeric             
n_tokens_title                 | Numeric             
n_tokens_content               | Numeric             
n_unique_tokens                | Numeric             
n_non_stop_words               | Numeric             
n_non_stop_unique_tokens       | Numeric             
num_hrefs                      | Numeric             
num_self_hrefs                 | Numeric             
num_imgs                       | Numeric             
num_videos                     | Numeric             
average_token_length           | Numeric             
num_keywords                   | Numeric             
data_channel_is_lifestyle      | Numeric             
data_channel_is_entertainment  | Numeric             
data_channel_is_bus            | Numeric             
data_channel_is_socmed    

Dropping unwanted columns and Splitting the dataset into train(80%) and test(20%)

In [8]:
df['shares']

0         593
1         711
2        1500
3        1200
4         505
         ... 
39639    1800
39640    1900
39641    1900
39642    1100
39643    1300
Name: shares, Length: 39644, dtype: int64

In [9]:
cols_to_drop = ['shares', 'url', 'timedelta']
# Ensure we only drop columns that actually exist in the dataframe
existing_cols_to_drop = [c for c in cols_to_drop if c in df.columns]

X = df.drop(existing_cols_to_drop, axis=1)

# y contains ONLY the target
y = df['shares']

# (Optional) If doing classification as suggested previously:
# y = (df[target_column] > 1400).astype(int)

# 3. Perform the Split
# test_size=0.2 means 20% of data goes to Test, 80% to Train.
# random_state=42 ensures the split is the same every time you run the code.
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.2, 
    random_state=42
)

In [10]:
X_train.shape

(31715, 58)

In [11]:
X_test.shape

(7929, 58)

In [12]:
print("X train", X_train.head())
print("X test", X_test.head())
print("y train", y_train.head())
print("y test",y_test.head())

X train        n_tokens_title  n_tokens_content  n_unique_tokens  n_non_stop_words  \
19837            15.0             628.0         0.509709               1.0   
904              11.0             409.0         0.519608               1.0   
23876             7.0             345.0         0.609329               1.0   
22966            11.0             217.0         0.580189               1.0   
4705             13.0             123.0         0.739837               1.0   

       n_non_stop_unique_tokens  num_hrefs  num_self_hrefs  num_imgs  \
19837                  0.645084       14.0             1.0       1.0   
904                    0.657480        2.0             0.0       1.0   
23876                  0.783920        5.0             2.0       1.0   
22966                  0.775000        8.0             2.0       3.0   
4705                   0.887324        3.0             0.0       1.0   

       num_videos  average_token_length  ...  avg_positive_polarity  \
19837         0.0  

In [13]:
def get_min_max_only(df):
    print(f"{'COLUMN NAME':<30} | {'MIN':<10} | {'MAX':<10}")
    print("-" * 60)
    
    for col in df.columns:
        # Only process numeric columns
        if pd.api.types.is_numeric_dtype(df[col]):
            min_val = df[col].min()
            max_val = df[col].max()
            
            # Print in a aligned row format
            print(f"{col:<30} | {min_val:<10} | {max_val:<10}")

# Run the function
get_min_max_only(X_train)

COLUMN NAME                    | MIN        | MAX       
------------------------------------------------------------
n_tokens_title                 | 2.0        | 23.0      
n_tokens_content               | 0.0        | 7413.0    
n_unique_tokens                | 0.0        | 701.0     
n_non_stop_words               | 0.0        | 1042.0    
n_non_stop_unique_tokens       | 0.0        | 650.0     
num_hrefs                      | 0.0        | 304.0     
num_self_hrefs                 | 0.0        | 116.0     
num_imgs                       | 0.0        | 128.0     
num_videos                     | 0.0        | 91.0      
average_token_length           | 0.0        | 8.04153354633
num_keywords                   | 1.0        | 10.0      
data_channel_is_lifestyle      | 0.0        | 1.0       
data_channel_is_entertainment  | 0.0        | 1.0       
data_channel_is_bus            | 0.0        | 1.0       
data_channel_is_socmed         | 0.0        | 1.0       
data_channel_is_tech    

Observation of min max value:

Observation 1: (cleaning required)
     4. n_unique_tokens:               Rate of unique words in the content
     5. n_non_stop_words:              Rate of non-stop words in the content
     6. n_non_stop_unique_tokens:      Rate of unique non-stop words in the content

These are rate columns, so the values cannot be more than 1.


Observation 2:(cleaning required)
    19. kw_min_min:                    Worst keyword (min. shares)
    20. kw_max_min:                    Worst keyword (max. shares)
    21. kw_avg_min:                    Worst keyword (avg. shares)
    22. kw_min_max:                    Best keyword (min. shares)
    23. kw_max_max:                    Best keyword (max. shares)
    24. kw_avg_max:                    Best keyword (avg. shares)
    25. kw_min_avg:                    Avg. keyword (min. shares)
    26. kw_max_avg:                    Avg. keyword (max. shares)
    27. kw_avg_avg:                    Avg. keyword (avg. shares)

These are share columns, so the values cannot be less Zero

Observation 3:(no cleaning required)
"Polarity" must be ranging from -1 to +1 
    -1 states it is negative polarity
    +1 states it is positive polarity

In [14]:
#Observation 1 : Treat the values more than 1

rate_cols = ['n_unique_tokens', 'n_non_stop_words', 'n_non_stop_unique_tokens']

print(f"{'COLUMN':<30} | {'ROWS > 1'}")
print("-" * 45)

for col in rate_cols:
    # 2. Count rows where the value is strictly greater than 1
    error_count = X_train[X_train[col] > 1].shape[0]
    
    print(f"{col:<30} | {error_count}")

# 3. View the actual bad data (optional)
# This lets you see if the value is something crazy like '700'
bad_data = X_train[X_train['n_unique_tokens'] > 1]
print("\nbad data rows are :")
print(bad_data[rate_cols])

COLUMN                         | ROWS > 1
---------------------------------------------
n_unique_tokens                | 1
n_non_stop_words               | 1
n_non_stop_unique_tokens       | 1

bad data rows are :
       n_unique_tokens  n_non_stop_words  n_non_stop_unique_tokens
31037            701.0            1042.0                     650.0


In [15]:
# List of rate columns to fix
rate_cols = ['n_unique_tokens', 'n_non_stop_words', 'n_non_stop_unique_tokens']

print(f"{'COLUMN':<30} | {'REPLACED':<8} | {'OLD MEAN':<10} | {'IMPUTED VAL':<11} | {'NEW MEAN':<10}")
print("-" * 80)

for col in rate_cols:
    # 1. Capture Old Mean (Includes the massive outliers > 1)
    old_mean = X_train[col].mean()
    
    # 2. Calculate the 'Clean Mean' (Exclude values > 1) -> This is our fill value
    clean_mean = X_train.loc[X_train[col] <= 1, col].mean()
    
    # 3. Identify and Count the bad rows
    mask_bad = X_train[col] > 1
    count_bad = mask_bad.sum()
    
    # 4. Replace bad values with the Clean Mean
    if count_bad > 0:
        X_train.loc[mask_bad, col] = clean_mean
        
        # 5. Calculate New Mean
        new_mean = X_train[col].mean()
        
        print(f"{col:<30} | {count_bad:<8} | {old_mean:<10.4f} | {clean_mean:<11.4f} | {new_mean:<10.4f}")
    else:
        print(f"{col:<30} | {'0':<8} | {old_mean:<10.4f} | {'N/A':<11} | {old_mean:<10.4f}")

# Verification step to ensure no values > 1 remain
print("\n" + "-"*80)
print("Verification (Should be 0 rows > 1):")
for col in rate_cols:
    remaining_errors = X_train[X_train[col] > 1].shape[0]
    print(f"{col:<30} | {remaining_errors}")

COLUMN                         | REPLACED | OLD MEAN   | IMPUTED VAL | NEW MEAN  
--------------------------------------------------------------------------------
n_unique_tokens                | 1        | 0.5520     | 0.5299      | 0.5299    
n_non_stop_words               | 1        | 1.0028     | 0.9700      | 0.9700    
n_non_stop_unique_tokens       | 1        | 0.6930     | 0.6725      | 0.6725    

--------------------------------------------------------------------------------
Verification (Should be 0 rows > 1):
n_unique_tokens                | 0
n_non_stop_words               | 0
n_non_stop_unique_tokens       | 0


In [16]:
# List of keyword (share) columns to check
kw_cols = [
    'kw_min_min', 'kw_max_min', 'kw_avg_min',
    'kw_min_max', 'kw_max_max', 'kw_avg_max',
    'kw_min_avg', 'kw_max_avg', 'kw_avg_avg'
]

total_rows = len(X_train)

print(f"{'COLUMN NAME':<30} | {'COUNT < 0':<10} | {'% OF TOTAL':<10}")
print("-" * 60)

# 1. Count rows and calculate percentage per column
for col in kw_cols:
    neg_count = X_train[X_train[col] < 0].shape[0]
    neg_pct = (neg_count / total_rows) * 100
    
    # Print with 2 decimal places for percentage
    print(f"{col:<30} | {neg_count:<10} | {neg_pct:.2f}%")

# 2. Show the top 5 rows that have negative values
condition = (X_train[kw_cols] < 0).any(axis=1)
bad_rows = X_train[condition]

print(f"\nTotal unique rows with at least one negative value: {bad_rows.shape[0]}")
print("\nTop 5 Rows with negative values:")
print(bad_rows[kw_cols].head(5))

COLUMN NAME                    | COUNT < 0  | % OF TOTAL
------------------------------------------------------------
kw_min_min                     | 18399      | 58.01%
kw_max_min                     | 0          | 0.00%
kw_avg_min                     | 652        | 2.06%
kw_min_max                     | 0          | 0.00%
kw_max_max                     | 0          | 0.00%
kw_avg_max                     | 0          | 0.00%
kw_min_avg                     | 6          | 0.02%
kw_max_avg                     | 0          | 0.00%
kw_avg_avg                     | 0          | 0.00%

Total unique rows with at least one negative value: 18399

Top 5 Rows with negative values:
       kw_min_min  kw_max_min  kw_avg_min  kw_min_max  kw_max_max  \
19837        -1.0       677.0  300.512857      7600.0    843300.0   
23876        -1.0       165.0   96.800000      7200.0    843300.0   
22966        -1.0       715.0  198.431667         0.0    843300.0   
36993        -1.0        49.0   11.500000   

"kw_min_min" has bad values more than ~58% so we will not consider that further analysis, rest of the columns - will replace it with mean

In [17]:
# List of columns to fix
kw_cols = [
    'kw_max_min', 'kw_avg_min',
    'kw_min_max', 'kw_max_max', 'kw_avg_max',
    'kw_min_avg', 'kw_max_avg', 'kw_avg_avg'
]

print(f"{'COLUMN':<25} | {'REPLACED':<8} | {'OLD MEAN':<10} | {'IMPUTED VAL':<11} | {'NEW MEAN':<10}")
print("-" * 75)

for col in kw_cols:
    # 1. Capture Old Mean (Includes the -1s, so it will be lower)
    old_mean = X_train[col].mean()
    
    # 2. Calculate the 'Clean Mean' (Exclude -1s) -> This is our fill value
    clean_mean = X_train.loc[X_train[col] >= 0, col].mean()
    
    # 3. Identify and Count the bad rows
    mask_neg = X_train[col] < 0
    count_neg = mask_neg.sum()
    
    # 4. Replace negative values with the Clean Mean
    if count_neg > 0:
        X_train.loc[mask_neg, col] = clean_mean
        
        # 5. Calculate New Mean (Should be equal to Clean Mean if all -1s are filled)
        new_mean = X_train[col].mean()
        
        print(f"{col:<25} | {count_neg:<8} | {old_mean:<10.2f} | {clean_mean:<11.2f} | {new_mean:<10.2f}")
    else:
        print(f"{col:<25} | {'0':<8} | {old_mean:<10.2f} | {'N/A':<11} | {old_mean:<10.2f}")

COLUMN                    | REPLACED | OLD MEAN   | IMPUTED VAL | NEW MEAN  
---------------------------------------------------------------------------
kw_max_min                | 0        | 1175.48    | N/A         | 1175.48   
kw_avg_min                | 652      | 316.06     | 322.72      | 322.72    
kw_min_max                | 0        | 13578.15   | N/A         | 13578.15  
kw_max_max                | 0        | 753332.38  | N/A         | 753332.38 
kw_avg_max                | 0        | 259157.53  | N/A         | 259157.53 
kw_min_avg                | 6        | 1120.29    | 1120.50     | 1120.50   
kw_max_avg                | 0        | 5681.05    | N/A         | 5681.05   
kw_avg_avg                | 0        | 3140.96    | N/A         | 3140.96   
